<a href="https://colab.research.google.com/github/Gauravkeny/Cats-and-Dogs-Prediction/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('Churn_Modelling.csv')

In [ ]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
del data['RowNumber']
del data['CustomerId']

In [ ]:
del data['Surname']

In [ ]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
data['Geography'].value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [ ]:
data_geog = pd.get_dummies(data['Geography'],prefix='Geography')

In [ ]:
data = pd.concat([data,data_geog],axis=1)
del data['Geography']

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,Female,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,Female,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [ ]:
data['Gender_Male'] = (data['Gender'] == 'Male').astype(int)

In [ ]:
del data['Gender']
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,0


In [ ]:
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV,cross_val_score

In [ ]:
xgb_params = {'n_estimators':[100,200,300,400,500,600,700,800,900,1000],
              'learning_rate':[0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05,0.1,0.5,1],
              'max_depth':[1,2,3,4,5,6]}

In [ ]:
X = data.drop(['Exited'],axis=1)
Y = data['Exited']

In [ ]:
X.dtypes

CreditScore            int64
Age                    int64
Tenure                 int64
Balance              float64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary      float64
Geography_France       uint8
Geography_Germany      uint8
Geography_Spain        uint8
Gender_Male            int64
dtype: object

In [ ]:
model = XGBClassifier()
xgb_random = RandomizedSearchCV(model,cv=10,n_iter=100,param_distributions=xgb_params,verbose=10000,scoring='roc_auc',n_jobs=-1)

In [ ]:
xgb_random.fit(X,Y)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   21.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   25.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, s...
                                           seed=None, silent=None, subsample=1,
                                           verbosity=1),
                   iid='deprecated', n_i

In [ ]:
xgb_random.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.005, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
Final_XGB_model = xgb_random.best_estimator_

In [ ]:
cv_xgb = cross_val_score(Final_XGB_model,X,Y,cv=10,scoring='roc_auc',verbose=10000)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  ................................................................
[CV] .................................... , score=0.880, total=   8.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.4s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.855, total=   8.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.9s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.882, total=   8.3s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   25.2s remaining:    0.0s
[CV]  ................................................................
[CV] .................................... , score=0.860, total=   8.3s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   33.5s remaining:    0.0s
[CV]  .........................

In [ ]:
cv_xgb.mean()

0.8661998168105441